## Idea of automizing the search for RNA-seq data within BioStudies (and ENA). 
### What it does: 
1. submits a query -> list of hits
2. filter hits for: organism, tissue (and more?)
3. a1) If data at ENA: check for sex and age in table
3. a2) If true: download fastq files
3. b1) If count matrix available at BioStudies -> download count matrix directly (check for raw counts)
### Learnings:
### - mostly ENA got the data (fastq (i need to calc it myself somewhen ...))
### - count matrix is attached to files 
### - probably more of this ... 

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import sys
from scipy.io import mmread
from utils.helper import get_negative_values, METADATA_COLS, reorder_columns_by_metadata_and_gene_counts, convert_gene_names_to_ensembl, clean_age
from utils.plotting import violinplot_overall, scatter_plot, manhattanplot
import requests
from utils.biostudies import *
from utils.ena import *


In [ ]:
url_get_study="https://www.ebi.ac.uk/biostudies/api/v1/studies/"

In [ ]:
%load_ext autoreload
%autoreload 2
###############################################################
#NOTE QUERIES!!!! 
###############################################################
# here i want to store some possible search queries, I produced the queries there and copied with inspect tool of the browser:  https://www.ebi.ac.uk/biostudies/ 
query_minimal = '%28RNA-seq+or+rnaseq%29+AND+%28human+or+%27homo+sapiens%27%29'   # ~3k results
query_medium = '%28RNA-seq+or+rnaseq%29+AND+%28human+or+%27homo+sapiens%27%29+AND+%28blood+OR+%27whole+blood%27+OR+PBMC%29+AND+NOT+%27single+cell%27+AND+NOT+%27cord+blood%27' # 630 results
query_long="%28rna-seq+OR+rnaseq%29+AND+%28human+OR+%22homo+sapiens%22%29+AND+%28blood+OR+%22whole+blood%22+OR+PBMC%29+AND+%28%22bulk%22%29+AND+NOT+%28sc+OR+%22single-cell%22%29+AND+NOT+DNA"
## pageSize 100 is max...

#NOTE used query! 
query = query_medium

current_page = 1
url_get_all = f"https://www.ebi.ac.uk/biostudies/api/v1/search?pageSize=100&page={str(current_page)}&query="

##
url = url_get_all + query
print(url)
data = get_biosample_query_response(url)
hits = data.get("hits", [])
total_hits = data.get("totalHits")

print(f'{len(hits)} / {total_hits}')

filtered_hits = [hit for hit in hits if "GEO" not in hit["accession"]]
## GEO Stuff will be handles by marc 

print(f'{len(filtered_hits)}')

# Display the filtered results
for hit in filtered_hits:
    
    response = requests.get(url_get_study + hit['accession'])
    if response.status_code == 200:
        data = response.json()
        
        is_useful, useful_files = filter_study_characteristics(data)
        print(f"Useful: {is_useful}")           
        if not is_useful:
            continue
    
        found_links = get_key_structure_for_string(data['section'], "links") # can be NoneType ... 
    
        print("="*50)            
        print(f"Accession: {hit['accession']}")
        print(f"Title: {hit['title']}")
        print(f"Views: {hit['views']}")
        print(f"Useful files: {useful_files} for {hit}")
        print(f"Found links: {found_links}")
        print("="*50)            

        if found_links:
            for found_link in found_links:
                link_block_handling(found_link)

    else:
        print(f"Failed to retrieve data: {response.status_code}")
    
    if total_hits - len(hits) > 0:
        current_page += 1
        
        #TODO next 100 pages ... when the first work fine :) 
        
        
    time.sleep(0.2)
